## Análisis de sentimientos

In [ ]:
import pandas as pd
import textblob
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
import csv


In [ ]:
df= pd.read_csv('isla de maipo25-03-2020_14-54-01.csv',header=0, sep=',')
comentarios=df['comentario' ]
#comentarios

In [ ]:
# Creamos una función para limpiar los comentarios
import re
def cleanTxt(text):
     text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
     text = re.sub('#', '', text) # Removing '#' hash tag
     text = re.sub('RT[\s]+', '', text) # Removing RT
     text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
     return text

# Limpiamos los comentarios
comentarios = comentarios.apply(cleanTxt)

# Mostramos los coemntarios limpios
comentarios

Queremos agregar la subjetividad y polaridad de los comentarios al dataframe. Para hacer esto, crearemos dos funciones:

Una para obtener los comentarios llamados subjetividad (cuán subjetivo u obstinado es el texto, un puntaje de 0 es un hecho y un puntaje de +1 es en gran medida una opinión) y
Otra para obtener el comentarios llamados Polarity (qué tan positivo o negativo es el texto, un puntaje de -1 es el puntaje negativo más alto y un puntaje de +1 es el puntaje positivo más alto). A continuación, almacenaremos los resultados en dos columnas, una llamada "Subjetividad" y la otra llamada "Polaridad" y mostraremos los resultados.

In [ ]:
df['polaridad']=df['comentario'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
#print(df['polaridad'].head(5))

In [ ]:
df['Subjectivity']=df['comentario'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [ ]:
print('polaridad')
print('valor maximo ',df['polaridad'].max())
print('valor minimo ',df['polaridad'].min())
print('valor medio ',df['polaridad'].mean())

In [ ]:
df

Veamos qué tan bien se distribuyen los sentimientos. Una buena manera de lograr esta tarea es entender las palabras comunes trazando nubes de palabras. Una nube de palabras (también conocida como nubes de texto o nubes de etiquetas) es una visualización, cuanto más aparece una palabra específica en el texto, más grande y más audaz aparece en la nube de palabras. Visualicemos todas las palabras en los datos usando el diagrama de nube de palabras. 

In [ ]:
# visualización de la nube de palabras
from wordcloud import WordCloud
allWords = ' '.join([comments for comments in df['comentario']])
wordCloud = WordCloud(width=500, height=300, random_state=21, max_font_size=110).generate(allWords)
plt.imshow(wordCloud, interpolation="bilinear")
plt.axis('off')
plt.show()

## Crearemos una función para calcular el análisis negativo (-1), neutral (0) y positivo (+1) y agregue la información a una nueva columna llamada Análisis, luego muestre los resultados.

In [ ]:
# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis
def getAnalysis(score):
 if score < 0:
  return 'Negative'
 elif score == 0:
  return 'Neutral'
 else:
  return 'Positive'


df['Analysis'] = df['polaridad'].apply(getAnalysis)

# Show the dataframe
df

Imprimamos los comentarios positivos en orden ascendente. El comentario más positivo es el comentario # 1.

In [ ]:
# Imprimimos los tweets positivos: 
print('Desplegamos los Comentarios positivos:\n')
j=1
sortedDF = df.sort_values(by=['polaridad']) # Ordenamos los comentarios
for i in range(0, sortedDF.shape[0] ):
  if( sortedDF['Analysis'][i] == 'Positive'):
    print(str(j) + ') '+ sortedDF['comentario'][i])
    print()
    j= j+1

Trace la polaridad y la subjetividad como un diagrama de dispersión. Parece que la mayoría de los comentarios son positivos, ya que muchos de los puntos están en el lado derecho de la polaridad en el valor 0.00.

In [ ]:
# Plotting 
plt.figure(figsize=(8,6)) 
for i in range(0, df.shape[0]):
  plt.scatter(df["polaridad"][i], df["Subjectivity"][i], color='Blue') # plt.scatter(x,y,color)   
plt.title('Sentiment Analysis') 
plt.xlabel('Polarity') 
plt.ylabel('Subjectivity') 
plt.show()

Imprimimos el porcentaje de comentarios positivos. Alrededor de 1/54 de los comentarios se consideran comentarios positivos.

In [ ]:
# Print the percentage of positive tweets
ptweets = df[df.Analysis == 'Positive']
ptweets = ptweets['comentario']
ptweets

round( (ptweets.shape[0] / df.shape[0]) * 100 , 1)

Imprime el porcentaje de comentarios negativos. Alrededor de 0/54 comentarios se consideran comentarios negativos.

In [ ]:
# Print the percentage of negative tweets
ntweets = df[df.Analysis == 'Negative']
ntweets = ntweets['comentario']
ntweets

round( (ntweets.shape[0] / df.shape[0]) * 100, 1)

Mostrar el valor cuenta. Ahora podemos ver que 81 comentarios son positivos, 10 son neutrales y 9 son negativos.

In [ ]:
# Show the value counts
df['Analysis'].value_counts()

Muestra visualmente el valor cuenta.

In [ ]:
# Trazar y visualizar los recuentos
plt.title('Análisis de sentimientos')
plt.xlabel('Sentimiento')
plt.ylabel('Recuentos')
df['Analysis'].value_counts().plot(kind = 'bar')
plt.show()

In [ ]:
keyword=input('Que concepto quieres buscar?: ')
numberoComentarios=int(input('Ingresa el número de Comentarios: '))

In [ ]:
#comentarios = (df[df['comentario']], keyword ).items(numberoComentarios)

positive = 0
negative = 0
neutral = 0
polarity = 0 

def calculatePercentage(a,b):
    return 100*float(a)/float(b)

for comentario in comentarios:
    print(comentario)
    myAnalysis=textblob.TextBlob(comentario)
    polarity += myAnalysis.sentiment.polarity
    if myAnalysis.sentiment.polarity ==0:
        neutral+=1
    elif myAnalysis.sentiment.polarity > 0.00:
        positive+=1
    elif myAnalysis.sentiment.polarity < 0.00:
        negative+=1

In [ ]:
labels = ['Positive [' + str(positive) + '%]', 'Neutral [' + str(neutral) + '%]','Negative [' + str(negative) + '%]']
sizes=[positive,neutral,negative]
colors=['green','yellow','red']
patches,texts=plt.pie(sizes,colors=colors,startangle=90)
plt.legend(patches,labels,loc="best")
plt.title('Cómo las personas reaccionan a ' + keyword + ' analizando ' + str(numberoComentarios) + ' comentarios.')
plt.axis('equal')
plt.tight_layout()
plt.show()